We would seek to control the alternating routing network so that it stays in the low-blocking state. One way to do this is with a technique called trunk reservation. We modify the call admission procedure, so that calls arising between $a$ and $b$ are routed as follows:
1.  Consider the direct link $a \leftrightarrow b$. If this link has spare capacity, then route the call over it.
2.  Otherwise, pick a new node $c$ uniformly at random from the other $K - 2$ nodes, and consider the two links $a \leftrightarrow c$ and $c \leftrightarrow b$. If on each of these links the number of calls in progress is less than $C - s$, then route the call over these two links.
3.  Otherwise, the call is blocked.
The parameter $s > 0$ is known as the trunk reservation parameter.

We first consider a single link in isolation and model the number of calls $i$ in progress on it. This link receives two types of traffic:

*   Primary traffic: This is traffic that originates on this link. We model this as a Poisson process with rate $\nu_1$. These calls are accepted as long as the link is not at full capacity, $i < C$.

*   Overflow traffic: This is traffic that has been rerouted from another link. We model this as an equivalent Poisson process with rate $\nu_2$. Due to trunk reservation, these calls are only accepted if the link has sufficient spare capacity, $i < C - s$.

The state of this single link is the number of calls $i \in \{0, \dots, C\}$ in progress. The departure rate from any state $i$ is $i$, as each call has a mean duration of $1$.

The difference is in the arrival rates for each state:
*   For $0 \leq i < C - s$, both primary and overflow calls are accepted. The total arrival rate is $\nu_1 + \nu_2$.
*   For $C - s \leq i < C$, only primary calls are accepted. The arrival rate is $\nu_1$.

Let $\pi_i$ be the equilibrium probability of having $i$ calls on the link. Solving the detailed balance equations for this Markov model gives the following relationships:
\begin{equation}
    \pi_i =
    \begin{cases}
         \frac{\pi_0(\nu_1 + \nu_2)^i}{i!} & 0 \leq i < C-s \\
         \frac{\pi_0(\nu_1 + \nu_2)^{C-s} \nu_1^{i-(C-s)}}{i!} & C-s \leq i \leq C
    \end{cases}
\end{equation}
The value of $\pi_0$ is found by ensuring that the sum of all the probabilities equals $1$.

From this single-link model, we can define two probabilities:
*   The probability that a primary call is blocked. This happens only when the link is completely full, $i = C$,
\begin{equation}
    P_{\text{direct}} = \pi_C.
\end{equation}
*   The probability that an overflow call is blocked. This happens when the link has $C-s$ or more calls in progress,
\begin{equation}
    P_{\text{overflow}} = \sum_{i=C-s}^C \pi_i.
\end{equation}

Now we embed this single-link model into the full network, creating a self-consistent system of equations.
*   The primary traffic rate $\nu_1$ is simply the given arrival rate $\nu$ for each pair of nodes,
\begin{equation}
    \nu_1 = \nu.
\end{equation}

*   The overflow traffic rate is the rate of calls that find their direct link busy, are directed to our link as part of an alternative path, and for which the other link in that alternative path is available,
    \begin{equation}
        \nu_2 = 2\nu P_{\text{direct}}(1 - P_{\text{overflow}}).
    \end{equation}

A fixed point is a set of values for these probabilities that are mutually consistent with the coupled system. Let $B$ be the probability that a new, incoming call is ultimately blocked, so its direct link is full and its chosen two-link alternative path is also unavailable,
\begin{equation}
    B = P_{\text{direct}} (1 - (1 - P_{\text{overflow}})^2).
\end{equation}
To find $B$, we must numerically solve for the consistent values of $P_{\text{direct}}$ and $P_{\text{overflow}}$ and then substitute them into this final formula.

In [9]:
import numpy as np
import math
from scipy.special import gammaln, logsumexp

def erlang_b_formula(nu, C):
    if C < 0 or not isinstance(C, int):
        raise ValueError("Capacity (C) must be a non-negative integer.")
    if nu < 0:
        return 0.0
    if C == 0:
        return 1.0 if nu > 0 else 0.0
    if nu == 0:
        return 0.0
    log_terms = [
        k * np.log(nu) - gammaln(k + 1) for k in range(C + 1)
    ]
    log_numerator = C * np.log(nu) - gammaln(C + 1)
    log_denominator = logsumexp(log_terms)
    if log_denominator == -np.inf:
        return 0.0

    log_blocking_probability = log_numerator - log_denominator
    blocking_probability = np.exp(log_blocking_probability)

    return blocking_probability

def calculate_pi_distribution_trunk(nu1, nu2, C, s):
    '''
    Calculates the equilibrium probability distribution π_i for a single link
    with two traffic classes and trunk reservation.
    '''
    C_int, s_int = int(C), int(s)

    # Calculate un-normalised terms iteratively
    terms = np.zeros(C_int + 1)
    terms[0] = 1.0

    # Region where both traffic types are accepted
    for i in range(1, C_int - s_int + 1):
        rate = nu1 + nu2
        terms[i] = terms[i-1] * rate / i

    # Region where only primary traffic is accepted
    for i in range(C_int - s_int + 1, C_int + 1):
        rate = nu1
        terms[i] = terms[i-1] * rate / i

    # Normalise to get the final distribution
    total_sum = np.sum(terms)
    if total_sum == 0:
        return np.zeros(C_int + 1)

    pi_distribution = terms / total_sum
    return pi_distribution

def solve_trunk_reservation_fp(nu, C, s, max_iter=100, tol=1e-12):
    '''
    Solves the fixed-point equations for the trunk reservation system iteratively.
    '''
    nu1 = nu
    p_direct_block, p_overflow_block = 0.0, 0.0  # Initial guess

    for _ in range(max_iter):
        # Calculate overflow rate from previous blocking probabilities
        nu2 = 2 * nu * p_direct_block * (1 - p_overflow_block)

        # Calculate new equilibrium distribution for a single link
        pi = calculate_pi_distribution_trunk(nu1, nu2, C, s)

        # Calculate new blocking probabilities from the distribution
        p_direct_new = pi[int(C)]
        p_overflow_new = np.sum(pi[int(C)-int(s):])

        # Check for convergence
        if abs(p_direct_new - p_direct_block) < tol and abs(p_overflow_new - p_overflow_block) < tol:
            break

        # Update for the next iteration
        p_direct_block, p_overflow_block = p_direct_new, p_overflow_new

    # After convergence, calculate the final overall blocking probability B
    prob_alt_path_fails = 1 - (1 - p_overflow_block)**2
    final_B = p_direct_block * prob_alt_path_fails

    return final_B


# System parameters and previous results
C_main = 600
nu_main = 560
prob_low_regime_no_res = 0.007548
prob_high_regime_no_res = 0.174550
prob_no_alt_routing = erlang_b_formula(nu_main, C_main)

s_values_to_test = [1, 10, 20, 40, 60, 80, 100]
results_table = {}

for s in s_values_to_test:
    blocking_prob = solve_trunk_reservation_fp(nu_main, C_main, s)
    results_table[s] = blocking_prob

print(f"{'System Configuration':<35} | {'Blocking Probability':>25}")
print("-" * 65)
print(f"{'Alt. Routing (Low Regime, s=0)':<35} | {prob_low_regime_no_res:>24.4%}")
print(f"{'Alt. Routing (High Regime, s=0)':<35} | {prob_high_regime_no_res:>24.4%}")
print(f"{'No Alt. Routing (Direct Only)':<35} | {prob_no_alt_routing:>24.4%}")
print("-" * 65)

for s_val, prob in results_table.items():
    print(f"{f'Trunk Reservation (s={s_val})':<35} | {prob:>24.4%}")

System Configuration                |      Blocking Probability
-----------------------------------------------------------------
Alt. Routing (Low Regime, s=0)      |                  0.7548%
Alt. Routing (High Regime, s=0)     |                 17.4550%
No Alt. Routing (Direct Only)       |                  0.4222%
-----------------------------------------------------------------
Trunk Reservation (s=1)             |                  0.0217%
Trunk Reservation (s=10)            |                  0.0997%
Trunk Reservation (s=20)            |                  0.1817%
Trunk Reservation (s=40)            |                  0.3312%
Trunk Reservation (s=60)            |                  0.4073%
Trunk Reservation (s=80)            |                  0.4214%
Trunk Reservation (s=100)           |                  0.4222%


The key improvement of trunk reservation is the elimination of the high-blocking regime so the system now has a single, stable, low blocking probability. A larger $s$ provides a stronger defense against congestion but diminishes the benefit of alternative routing. As $s$ grows, the performance approaches the baseline of direct routing only. A smaller $s$ maximises the performance gain from rerouting but may be less robust if the primary traffic load increases.